# Goal of this notebook: call somatic mutations and implement locus-level filters

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context("poster")

%matplotlib inline

# Load STR genotypes, call somatic mutations

In [33]:
df_patient_gts = pd.read_csv("../../data/dummy_STR_mutation_data.csv")
df_patient_gts


,patient,tmp_id,period,ref,allele_a_healthy,allele_a_tumor,allele_b_healthy,allele_b_tumor,MSI
0,1,chr1_10008054,1,14,14,14,14,14,MSS
1,1,chr1_10014276,2,4,4,4,4,4,MSS
2,1,chr1_1004515,2,4,4,4,4,4,MSS
3,1,chr1_10054784,1,19,19,19,19,19,MSS
4,1,chr1_10059809,1,9,9,9,9,9,MSS
...,...,...,...,...,...,...,...,...,...
49995,50,chr1_9981868,1,13,13,13,13,13,MSI
49996,50,chr1_9982962,2,4,4,4,4,3,MSI
49997,50,chr1_999504,3,4,4,4,4,4,MSI
49998,50,chr1_9997162,2,4,4,4,4,4,MSI


In [34]:
def calc_len_differences(a: np.array, b: np.array, aprime: np.array, bprime: np.array):
    """
    Given four arrays of STR allele lengths (two from healthy samples, two from patient-matched tumor samples),
    calculate the smallest unit number difference that would transform  'a' and 'b' into 'a_prime' and 'b_prime'.
    To do this, calculate the difference for two scenarios:
        a -> a_prime & b -> b_prime 
        OR 
        a -> b_prime & b -> a_prime
    The scenario yielding the smallest unit difference between the alleles of the healthy sample and the tumor
    sample is assumed to be the most likely.
    """
    # make matrix to store results
    # res_m[0]: alle_a_differences, res_m[1]: allele_b_differences, res_m[2]: sum of absolute differences
    res_m = np.array(
        [
            np.zeros(len(a)),
            np.zeros(len(a)),
            np.zeros(len(a)),
        ]    
    )
    # Case where allele a healthy -> allele a tumor
    a_to_a = np.array([aprime-a, bprime-b])
    a_to_a_diff = np.abs(a_to_a).sum(axis=0)
    # Case where allele a healthy -> allele b tumor
    a_to_b = np.array([bprime-a, aprime-b])
    a_to_b_diff = np.abs(a_to_b).sum(axis=0)
    
    # Make boolian array for indexing, smallest difference between healthy and 
    # tumor is assumed most likely scenario
    comp = (a_to_a_diff >= a_to_b_diff)
    
    res_m[0:2, ~comp] = a_to_a[:, ~comp]
    res_m[0:2, comp] = a_to_b[:, comp]
    
    res_m[2, ~comp] = a_to_a_diff[~comp]
    res_m[2, comp] = a_to_b_diff[comp]
    
    return res_m.T

In [35]:
res = calc_len_differences(
    df_patient_gts.allele_a_healthy.to_numpy(),
    df_patient_gts.allele_b_healthy.to_numpy(),
    df_patient_gts.allele_a_tumor.to_numpy(),
    df_patient_gts.allele_b_tumor.to_numpy(),   
)

df_patient_gts[['allele_a_diff', 'allele_b_diff', 'patient_len_diff']] = pd.DataFrame(res, index=df_patient_gts.index).astype(np.int32)
df_patient_gts


,patient,tmp_id,period,ref,allele_a_healthy,allele_a_tumor,allele_b_healthy,allele_b_tumor,MSI,allele_a_diff,allele_b_diff,patient_len_diff
0,1,chr1_10008054,1,14,14,14,14,14,MSS,0,0,0
1,1,chr1_10014276,2,4,4,4,4,4,MSS,0,0,0
2,1,chr1_1004515,2,4,4,4,4,4,MSS,0,0,0
3,1,chr1_10054784,1,19,19,19,19,19,MSS,0,0,0
4,1,chr1_10059809,1,9,9,9,9,9,MSS,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
49995,50,chr1_9981868,1,13,13,13,13,13,MSI,0,0,0
49996,50,chr1_9982962,2,4,4,4,4,3,MSI,-1,0,1
49997,50,chr1_999504,3,4,4,4,4,4,MSI,0,0,0
49998,50,chr1_9997162,2,4,4,4,4,4,MSI,0,0,0


In [36]:
df_patient_gts.query("patient_len_diff != 0")

,patient,tmp_id,period,ref,allele_a_healthy,allele_a_tumor,allele_b_healthy,allele_b_tumor,MSI,allele_a_diff,allele_b_diff,patient_len_diff
65,1,chr1_10372810,4,3,3,3,3,4,MSS,1,0,1
66,1,chr1_10375386,3,7,7,5,7,7,MSS,0,-2,2
69,1,chr1_10404523,1,13,13,16,13,13,MSS,0,3,3
104,1,chr1_10587445,4,4,4,1,4,4,MSS,0,-3,3
113,1,chr1_10659522,2,5,5,7,5,5,MSS,0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...
49947,50,chr1_9725046,2,4,4,12,4,4,MSI,0,8,8
49983,50,chr1_9930713,4,5,5,4,5,5,MSI,0,-1,1
49985,50,chr1_9932102,1,16,16,13,16,16,MSI,0,-3,3
49989,50,chr1_9972495,1,15,15,15,15,14,MSI,-1,0,1


## Filter out calls where patient is homozygous for an allele not observed in the matched healthy sample

See Methods section of Mitra <i>et al.</i>, (2021), under subheader 'Identifying de novo TR mutations': 

"We further filtered calls for which the child was homozygous for the new allele (n = 214,639) ..."

Reasoning seems to be to filter out cases where a heterozygous locus is spuriously called as homozygous due to drop out of an allele in one (or both) of the samples. We also see this type of mutation is extremely common in our data, even in MSS patients where this would not necessarily be expected.

Mitra, I., Huang, B., Mousavi, N. <i>et al.</i> Patterns of de novo tandem repeat mutations and their role in autism. <i>Nature</i> 589, 246–250 (2021). https://doi.org/10.1038/s41586-020-03078-7

In [37]:
print(df_patient_gts.shape)
print(df_patient_gts.query("patient_len_diff != 0").shape)
print()

mask = ~((df_patient_gts.allele_a_tumor == df_patient_gts.allele_b_tumor) &   # homozygous in tumor
        (df_patient_gts.allele_a_tumor != df_patient_gts.allele_a_healthy) &  # not equal healthy allele a
        (df_patient_gts.allele_a_tumor != df_patient_gts.allele_b_healthy))   # ...or healthy allele b

df_patient_gts_filt = df_patient_gts[mask]

print(df_patient_gts_filt.shape)
print(df_patient_gts_filt.query("patient_len_diff != 0").shape)
print(df_patient_gts.shape[0] - df_patient_gts_filt.shape[0])


(50000, 12)
(3241, 12)

(49977, 12)
(3218, 12)
23


In [38]:
df_patient_gts_filt.to_csv("../../data/dummy_somatic_mutation_calls_filt.csv", index=False)